<a href="https://colab.research.google.com/github/Jane0731/Homework/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A_W12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install openai pandas yfinance requests matplotlib nltk

In [7]:
!pip install --upgrade transformers sympy

In [33]:
from openai import OpenAI

# 設定 OpenAI API 金鑰 -> 有成功 但為了資安 所以拔除 不上傳 github
os.environ["OPENAI_API_KEY"] = "隱藏"


# 初始化 OpenAI 客戶端
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

# 基本面分析

In [51]:
import yfinance as yf

def get_fundamental_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    fundamental_data = {
        "公司名稱": info.get("longName"),
        "行業": info.get("sector"),
        "市盈率(P/E)": info.get("trailingPE"),
        "股息率": info.get("dividendYield"),
        "總收入": info.get("totalRevenue"),
        "淨利潤率": info.get("profitMargins"),
    }
    return fundamental_data

def format_fundamental_data(fundamental_data):
    # 格式化數字，處理 None 或數據缺失的情況
    pe_ratio = fundamental_data.get("市盈率(P/E)", "N/A")
    dividend_yield = fundamental_data.get("股息率", "N/A")
    total_revenue = fundamental_data.get("總收入", "N/A")
    profit_margin = fundamental_data.get("淨利潤率", "N/A")

    # 將大數據轉換為可讀的單位（例如 "2000億"）
    if total_revenue and isinstance(total_revenue, int):
        total_revenue = f"{total_revenue / 1e8:.1f}億"

    if profit_margin and isinstance(profit_margin, float):
        profit_margin = f"{profit_margin * 100:.1f}%"

    # 格式化輸出
    formatted_data = (
        f"市盈率(P/E Ratio): {pe_ratio}, "
        f"股息率: {dividend_yield}, "
        f"營收: {total_revenue}, "
        f"淨利潤率: {profit_margin}"
    )
    return formatted_data


# 技術面分析

In [46]:
import yfinance as yf
import pandas as pd
import openai
import os

def calculate_technical_indicators(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="3mo")  # 最近三個月的數據

    # 計算技術指標
    hist['20_MA'] = hist['Close'].rolling(window=20).mean()
    hist['RSI'] = 100 - (100 / (1 + hist['Close'].pct_change().apply(lambda x: x if x > 0 else 0).rolling(14).mean() / hist['Close'].pct_change().apply(lambda x: abs(x)).rolling(14).mean()))

    # 提取最新數據
    latest_data = {
        "最新價格": hist['Close'][-1],
        "20日均線": hist['20_MA'][-1],
        "RSI": hist['RSI'][-1]
    }
    return latest_data



def ask_gpt_about_technical_analysis(ticker):
    technical_data=calculate_technical_indicators(ticker)
    # 格式化技術面數據
    close = technical_data.get("最新價格", "N/A")
    rsi = technical_data.get("RSI", "N/A")
    ma_20 = technical_data.get("20_MA", "N/A")
    # 確保數據為數字，否則轉為 "N/A"
    close = f"{close:.2f}" if isinstance(close, (int, float)) else "N/A"
    rsi = f"{rsi:.2f}" if isinstance(rsi, (int, float)) else "N/A"
    ma_20 = f"{ma_20:.2f}" if isinstance(ma_20, (int, float)) else "N/A"

    # 構建 Prompt
    prompt = f"""
    以下是 {ticker} 的技術面數據：
    RSI: {rsi}, 20日均線: {ma_20}, 最新價格: {close}

    請基於這些數據，分析該股票的技術面情況並提供投資建議。
    建議不用太長，不要超過 100 個字，請給重點
    """

    # 使用 Chat API
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "你是專業的股票分析師，提供詳細且準確的技術分析和投資建議。",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        max_tokens=500,
    )
    gpt_response = response.choices[0].message.content
    return gpt_response

# 新聞情緒分析模組

In [49]:
import requests
from transformers import pipeline

# 初始化情感分析模型
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="uer/roberta-base-finetuned-chinanews-chinese"
)

def fetch_news(stock_id, page=1, limit=5):
    url = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock'
    params = {
        'page': page,
        'limit': limit,
        'stock_id': stock_id
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# 獲取台積電（2330.TW）的新聞資料並進行情感分析
def analyze_sentiment(stock_id, page=1, limit=5):
    news_data = fetch_news(stock_id, page, limit)
    if news_data:
        articles = news_data['items']['data']
        sentiment_scores = []
        for news in articles:
            title = news.get('title', '無標題')
            sentiment = sentiment_pipeline(title)[0]  # 分析情感
            print(f"標題: {title}")
            print(f"情感結果: {sentiment}")
            print(f"情感傾向: {sentiment['label']} ({'正面' if sentiment['label'] == 'positive' else '負面'})")
            print()
            sentiment_scores.append(sentiment['score'] if sentiment['label'] == 'positive' else -sentiment['score'])

        # 計算新聞數量及平均情緒分數
        avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0
        sentiment_analysis = {
            "新聞數量": len(articles),
            "平均情緒分數": avg_sentiment
        }
        return sentiment_analysis
    else:
        print("無法獲取新聞資料")
        return None

def summarize_sentiment_to_gpt(stock_id, sentiment_analysis):
    # 整合數據為文字
    news_count = sentiment_analysis.get("新聞數量", 0)
    avg_sentiment = sentiment_analysis.get("平均情緒分數", 0)

    prompt = f"""
    以下是 {stock_id} 的新聞情緒分析摘要：
    - 總共分析的新聞數量：{news_count} 篇
    - 平均情緒分數：{avg_sentiment:.2f}

    請基於這些數據生成一個簡短的敘述，描述新聞情緒的總體趨勢（例如偏向正面、負面或中性）。
    """

    # 呼叫 GPT API
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "你是一位專業的金融分析師，能根據情緒數據生成簡明扼要的分析報告。",
                },
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            max_tokens=150,
        )
        gpt_response = response.choices[0].message.content
        return gpt_response

    except Exception as e:
        print(f"API 調用失敗: {e}")
        return None


標題: 聯電「綠獎」推動生物多樣性 九年累積81項成果遍及全台
情感結果: {'label': 'Hong Kong - Macau politics', 'score': 0.9968279004096985}
情感傾向: Hong Kong - Macau politics (負面)

標題: 〈航運指數〉歐洲運價暴漲 但美西暴跌 貨攬業點出一原因
情感結果: {'label': 'financial news', 'score': 0.8462272882461548}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：兆豐金(2886-TW)EPS預估下修至2.47元，預估目標價為39.6元
情感結果: {'label': 'financial news', 'score': 0.997488260269165}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：富邦金(2881-TW)EPS預估上修至9.61元，預估目標價為98元
情感結果: {'label': 'financial news', 'score': 0.9983288645744324}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：保瑞(6472-TW)EPS預估上修至41.93元，預估目標價為950元
情感結果: {'label': 'financial news', 'score': 0.9975008368492126}
情感傾向: financial news (負面)

GPT 回應：
根據我們對2330的新聞情緒分析，總共納入的新聞篇數為5篇。自這些數據中，我們可以觀察到情緒得分呈現出負向的趨勢，平均情緒分數為-0.97，代表該檔股票的新聞情緒偏向負面，可能對其市場表現造成不利影響。


# 使用 OpenAI 生成報告

In [54]:
import os



def generate_stock_report(ticker, fundamental, technical, sentiment):
    prompt = f"""
    以下是 {ticker} 的股票數據分析：
    基本面數據：
    {fundamental}

    技術面數據：
    {technical}

    新聞情緒：
    {sentiment}

    請基於上述數據生成一份專業的股票分析報告，包括投資建議和風險提示。
    請給我 500 字以內的報告
    """
    # 使用 ChatCompletions API
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 或其他支持的 GPT 模型
        messages=[
            {
                "role": "system",
                "content": "你是專業的股票分析師，提供詳細且準確的報告。",
            },
            {
                "role": "user",
                "content": prompt
            },
        ],
        max_tokens=1000,
    )
    gpt_response = response.choices[0].message.content
    return gpt_response

<ipython-input-46-f8242bd06398>:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "最新價格": hist['Close'][-1],
<ipython-input-46-f8242bd06398>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "20日均線": hist['20_MA'][-1],
<ipython-input-46-f8242bd06398>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "RSI": hist['RSI'][-1]


標題: 聯電「綠獎」推動生物多樣性 九年累積81項成果遍及全台
情感結果: {'label': 'Hong Kong - Macau politics', 'score': 0.9968279004096985}
情感傾向: Hong Kong - Macau politics (負面)

標題: 〈航運指數〉歐洲運價暴漲 但美西暴跌 貨攬業點出一原因
情感結果: {'label': 'financial news', 'score': 0.8462272882461548}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：兆豐金(2886-TW)EPS預估下修至2.47元，預估目標價為39.6元
情感結果: {'label': 'financial news', 'score': 0.997488260269165}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：富邦金(2881-TW)EPS預估上修至9.61元，預估目標價為98元
情感結果: {'label': 'financial news', 'score': 0.9983288645744324}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：保瑞(6472-TW)EPS預估上修至41.93元，預估目標價為950元
情感結果: {'label': 'financial news', 'score': 0.9975008368492126}
情感傾向: financial news (負面)

# 股票分析報告 - 2330.TW

## 基本面分析
2330.TW目前的市盈率（P/E Ratio）為24.59，這表明公司在市場上相對於其盈利能力的估值偏高。此外，0.0159的股息率顯示出其回報相對較低，可能不太吸引尋求穩定現金流的投資者。營收方面，26513.8億元的數字顯示出公司在行業中的強大地位，然而，相對於39.1%的淨利潤率，這仍需關注其持續的盈利能力和競爭壓力。

## 技術面分析
從技術面來看，2330.TW的相對強弱指數（RSI）為21.28，顯示出該股票目前處於超賣狀態。這通常暗示著價格可能會反彈。然而，市場情緒和其他影響因素

# 投資建議排序

In [55]:
def analyze_and_rank_stocks(tickers):
    reports = []
    for ticker in tickers:
        fundamental_data=get_fundamental_data(ticker)
        fundamental = format_fundamental_data(fundamental_data)
        technical = ask_gpt_about_technical_analysis(ticker)
        sentiment_analysis = analyze_sentiment(ticker)
        sentiment = summarize_sentiment_to_gpt(ticker, sentiment_analysis)
        report = generate_stock_report(ticker, fundamental, technical, sentiment)
        reports.append({"股票": ticker, "報告": report})

    # 根據您的需求設計排序邏輯（例如 P/E、RSI、新聞情緒分數等）
    return sorted(reports, key=lambda x: x['股票'])

# 示例
tickers = ["2330.TW", "AAPL", "MSFT"]
ranked_reports = analyze_and_rank_stocks(tickers)
for stock in ranked_reports:
    print(stock)

<ipython-input-46-f8242bd06398>:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "最新價格": hist['Close'][-1],
<ipython-input-46-f8242bd06398>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "20日均線": hist['20_MA'][-1],
<ipython-input-46-f8242bd06398>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "RSI": hist['RSI'][-1]


標題: 聯電「綠獎」推動生物多樣性 九年累積81項成果遍及全台
情感結果: {'label': 'Hong Kong - Macau politics', 'score': 0.9968279004096985}
情感傾向: Hong Kong - Macau politics (負面)

標題: 〈航運指數〉歐洲運價暴漲 但美西暴跌 貨攬業點出一原因
情感結果: {'label': 'financial news', 'score': 0.8462272882461548}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：兆豐金(2886-TW)EPS預估下修至2.47元，預估目標價為39.6元
情感結果: {'label': 'financial news', 'score': 0.997488260269165}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：富邦金(2881-TW)EPS預估上修至9.61元，預估目標價為98元
情感結果: {'label': 'financial news', 'score': 0.9983288645744324}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：保瑞(6472-TW)EPS預估上修至41.93元，預估目標價為950元
情感結果: {'label': 'financial news', 'score': 0.9975008368492126}
情感傾向: financial news (負面)



<ipython-input-46-f8242bd06398>:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "最新價格": hist['Close'][-1],
<ipython-input-46-f8242bd06398>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "20日均線": hist['20_MA'][-1],
<ipython-input-46-f8242bd06398>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "RSI": hist['RSI'][-1]


標題: 聯電「綠獎」推動生物多樣性 九年累積81項成果遍及全台
情感結果: {'label': 'Hong Kong - Macau politics', 'score': 0.9968279004096985}
情感傾向: Hong Kong - Macau politics (負面)

標題: 〈航運指數〉歐洲運價暴漲 但美西暴跌 貨攬業點出一原因
情感結果: {'label': 'financial news', 'score': 0.8462272882461548}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：兆豐金(2886-TW)EPS預估下修至2.47元，預估目標價為39.6元
情感結果: {'label': 'financial news', 'score': 0.997488260269165}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：富邦金(2881-TW)EPS預估上修至9.61元，預估目標價為98元
情感結果: {'label': 'financial news', 'score': 0.9983288645744324}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：保瑞(6472-TW)EPS預估上修至41.93元，預估目標價為950元
情感結果: {'label': 'financial news', 'score': 0.9975008368492126}
情感傾向: financial news (負面)



<ipython-input-46-f8242bd06398>:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "最新價格": hist['Close'][-1],
<ipython-input-46-f8242bd06398>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "20日均線": hist['20_MA'][-1],
<ipython-input-46-f8242bd06398>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "RSI": hist['RSI'][-1]


標題: 聯電「綠獎」推動生物多樣性 九年累積81項成果遍及全台
情感結果: {'label': 'Hong Kong - Macau politics', 'score': 0.9968279004096985}
情感傾向: Hong Kong - Macau politics (負面)

標題: 〈航運指數〉歐洲運價暴漲 但美西暴跌 貨攬業點出一原因
情感結果: {'label': 'financial news', 'score': 0.8462272882461548}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：兆豐金(2886-TW)EPS預估下修至2.47元，預估目標價為39.6元
情感結果: {'label': 'financial news', 'score': 0.997488260269165}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：富邦金(2881-TW)EPS預估上修至9.61元，預估目標價為98元
情感結果: {'label': 'financial news', 'score': 0.9983288645744324}
情感傾向: financial news (負面)

標題: 鉅亨速報 - Factset 最新調查：保瑞(6472-TW)EPS預估上修至41.93元，預估目標價為950元
情感結果: {'label': 'financial news', 'score': 0.9975008368492126}
情感傾向: financial news (負面)

{'股票': '2330.TW', '報告': '### 股票分析報告：2330.TW\n\n#### 基本面分析\n2330.TW（台積電）目前的市盈率為24.59，這一數據顯示出市場對於其未來盈利的高期望。在股息率為0.0159的背景下，投資者可獲得一定的現金流入。公司年營收達到26513.8億元，淨利潤率高達39.1%，顯示出其良好的盈利能力與市場地位。然而，需密切關注公司的業績報告及未來增長前景，以預測其長期表現。\n\n#### 技術面分析\n技術面上，RSI指標目前為21.28，明顯低於30的超賣臨界值，暗示該股在短期內可能存在反彈的機會。儘